[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fernigithub/series_temporales_101/blob/main/notebooks/NDVI_Modis.ipynb)

In [ ]:
# Import required libraries
import geemap
import ee
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import datetime
import ipyleaflet
import pandas as pd

try:
    # Initialize the Earth Engine module
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='earthengine-legacy')

# Get today's date
today = datetime.datetime.today().strftime('%Y-%m-%d')

# Filter collection to dates of interest
MOD = ee.ImageCollection('MODIS/061/MOD13Q1').filterDate('2005-01-01', today)

# Create a collection to sample from
EVI = MOD.select(['EVI']).map(lambda image: image.divide(10000).copyProperties(image, ['system:time_start']))

# Get the latest EVI image
EVI_latest = EVI.sort('system:time_start', False).first()

# Get the date of the latest EVI image
date_latest = ee.Date(EVI_latest.get('system:time_start')).format('dd-MM-yyyy').getInfo()

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 1,
    'palette': [
        'FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401',
        '056201', '004C00', '023B01', '012E01', '011301'
    ]
}

# Create a map
Map = geemap.Map(center=[-31.7667, -60.4893], zoom=10)
Map.addLayer(EVI_latest, vis_params, 'EVI MODIS ' + date_latest)

# Create a panel to hold our widgets
panel = widgets.VBox()
intro = widgets.VBox([
    widgets.HTML(value='<h3>MODIS MOD13Q1 EVI 01-01-2005 / ' + ee.Date(today).format('dd-MM-yyyy').getInfo() + '</h3>'),
    widgets.HTML(value='<p>Click on the map.</p>')
])
panel.children += (intro,)

# Create panels to hold lon/lat values
lon_label = widgets.Label()
lat_label = widgets.Label()
coords_panel = widgets.VBox([lon_label, lat_label])
panel.children += (coords_panel,)



def plot_evi_time_series(lon, lat):
    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)
    
    # Extract the EVI values for the point across the time series
    evi_time_series = EVI.getRegion(point, 500).getInfo()
    
    # Convert to a DataFrame
    header = evi_time_series[0]
    data = evi_time_series[1:]
    df = pd.DataFrame(data, columns=header)
    
    # Convert the time information to a readable format
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df['time'], df['EVI'], label='EVI')
    ax.set_title('MODIS EVI Time Series')
    ax.set_xlabel('Date')
    ax.set_ylabel('EVI')
    plt.legend()
    plt.show()
    

# Function to handle map click event
def on_click( **kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        lon_label.value = f'long: {lon:.2f}'
        lat_label.value = f'lat: {lat:.2f}'
        
        # Add a red dot for the point clicked on
        point = ee.Geometry.Point([lon, lat])
        marker = ipyleaflet.Marker(location=(lat, lon))
        Map.add_layer(marker)
        
        # Create an EVI chart
        plot_evi_time_series(lon, lat)
        

# Register the callback on the map
Map.on_interaction(on_click)

# Display the map and panel
display(panel)
Map.add_layer_control()
Map